In [23]:
import folium
import geopandas as gpd
import rasterio as rio
from osgeo import osr, gdal

In [85]:
attr_Esri_WorldImagery = 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
tiles_Esri_WorldImagery = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

val_riv_joined = gpd.read_file('data_github/shp/HWM_joined.shp')
val_surge_joined = gpd.read_file('data_github/shp/Peak_water_level_joined.shp')
neuse_huc6 = gpd.read_file('data_github/shp/nc_huc6.shp')

# Create a geometry list from the GeoDataFrame
val_riv_joined_geom = [[point.xy[1][0], point.xy[0][0]] for point in val_riv_joined.geometry]
val_surge_joined_geom = [[point.xy[1][0], point.xy[0][0]] for point in val_surge_joined.geometry]

m = folium.Map(
    location=[35.3, -77.5],
    zoom_start=8.5,
    tiles=tiles_Esri_WorldImagery,
    attr=attr_Esri_WorldImagery
    )

# iterate through list and add markers and labels for validation points
for i, coordinates in enumerate(val_riv_joined_geom):
    m.add_child(
        folium.Marker(
            location = coordinates,
            popup = "HWM" + '<br>' +
            "El (ft): " + str(val_riv_joined.elev_ft[i]),
            icon = folium.Icon(color = 'green')
            )
            )

geo_j = neuse_huc6.to_crs(epsg=4326).to_json() # folium geographic crs
geo_j = folium.GeoJson(data=geo_j)
folium.Popup('Neuse HUC 030202').add_to(geo_j)
geo_j.add_to(m)

In [86]:
# show map in notebook
m